In [23]:
import os
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import cv2
# from evaluator import ModelEvaluator
from tqdm import tqdm
from torchsummary import summary
from fvcore.nn import FlopCountAnalysis, parameter_count
from ptflops import get_model_complexity_info
import time

In [24]:
# Device 설정
device = torch.device("cpu")
print(f"Using device: {device}")

Using device: cpu


In [25]:
# 데이터셋 경로
data_dir = "data"

In [26]:
# 전처리
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet stats
])

In [27]:
# 데이터셋 로드
dataset = ImageFolder(root=data_dir, transform=transform)
# dataset = ImageFolder(root=data_dir)

# 클래스 정보 출력
print(f"Classes: {dataset.classes}")

Classes: ['with_mask', 'without_mask']


In [28]:
# Train:Val:Test = 70:15:15 분할
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_set, val_set, test_set = random_split(dataset, [train_size, val_size, test_size])


In [29]:
# DataLoader 생성
batch_size = 16
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [30]:
# CNN 기반 마스크 착용 여부 이진 분류 모델
class MaskClassifier(nn.Module):
    def __init__(self):
        super(MaskClassifier, self).__init__()
        
        # Feature Extraction - 더 얕은 구조로 변경
        self.features = nn.Sequential(
            # First Block
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Dropout2d(0.2),
            
            # Second Block
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Dropout2d(0.2),
            
            # Third Block
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Dropout2d(0.2),
        )
        
        # Classifier - 더 단순한 구조로 변경
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(128, 2)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [31]:
model = MaskClassifier().to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()

# Learning rate와 optimizer 수정
learning_rate = 0.001  # 더 작은 learning rate 사용
num_epoch = 20  # epoch 수 감소

optimizer = optim.Adam(
    model.parameters(),
    lr=learning_rate,
    betas=(0.9, 0.999),
    weight_decay=0.0001  # 더 작은 weight decay
)

In [33]:
def train_model(model, train_loader, val_loader, epochs=10):
    best_val_acc = 0.0
    
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        correct = 0
        total = 0
        
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs}", unit="batch")
        
        for images, labels in progress_bar:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            # Progress Bar에 현재 배치의 accuracy 표시
            batch_acc = 100. * correct / total
            progress_bar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'acc': f'{batch_acc:.2f}%'
            })
    
    # 최종 학습 결과 평가
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    final_acc = 100. * correct / total
    print("\n=== Final Training Results ===")
    print(f"Final Validation Accuracy: {final_acc:.2f}%")

In [ ]:
# 학습 실행
train_model(model, train_loader, val_loader, num_epoch)

Epoch 2/20:  69%|██████▉   | 100/144 [00:53<00:23,  1.84batch/s, loss=0.2595, acc=86.44%]

In [ ]:
# 테스트 함수
def test_model(model, test_loader):
    """
    Args:
        model (torch.nn.Module): 평가할 모델
        test_loader (DataLoader): 테스트 데이터 로더
    """
    model.eval()
    all_labels = []
    all_preds = []

    with torch.no_grad():
        progress_bar = tqdm(test_loader, desc="Testing", unit="batch")  # Progress Bar 추가
        for images, labels in progress_bar:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

            # Progress Bar 상태 업데이트 (현재 배치의 예측 결과 일부 표시)
            progress_bar.set_postfix(batch_accuracy=(preds == labels).float().mean().item())

    print("\nTest Classification Report:")
    print(classification_report(all_labels, all_preds, target_names=dataset.classes))


In [ ]:
# 테스트 실행
model = MaskClassifier().to(device)
model.load_state_dict(torch.load("mask_classifier.pth", map_location=device))
model.eval()
test_model(model, test_loader)

Testing: 100%|██████████| 31/31 [00:10<00:00,  3.02batch/s, batch_accuracy=1]    


Test Classification Report:
              precision    recall  f1-score   support

   with_mask       0.95      0.94      0.94       243
without_mask       0.94      0.95      0.95       249

    accuracy                           0.95       492
   macro avg       0.95      0.95      0.95       492
weighted avg       0.95      0.95      0.95       492



In [ ]:
# 모델 저장 코드
def save_model(model, path="mask_classifier.pth"):
    torch.save(model.state_dict(), path)
    print(f"Model saved to {path}")

In [ ]:
# 학습 후 모델 저장 및 평가
save_model(model, "mask_classifier.pth")